In [1]:
# 국토교통부 버스노선(서울, 부산 제외)
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from ServiceKey import public_serviceKey,mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)
citycode = '34330'

# DB에서 도시 코드 구해오기
# 2021.09.23 데이터 삽입완료
query = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
b_cnt = 0
for j in location_list:
    location = j
    if b_cnt == 1:
        mydb.close()
        break
    
    for i in range(1,121):
        routeNo = i
        print("location : {} | {}번째 조회시작".format(location[0],routeNo))    
        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteNoList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('routeNo') : routeNo,
            quote_plus('numOfRows') : '10000'
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location : {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],routeNo))
            b_cnt += 1
            break
        if rows == []:
            print("{} 번쨰데이터가없습니다.".format(routeNo))
            continue
        for j in rows:
            item = j
            routeno = item.findAll('routeno')
            if routeno == []:
                routeno = None
            else:
                routeno = routeno[0].text

            routeid = item.findAll('routeid')
            if routeid == []:
                routeid = None
            else:
                routeid = routeid[0].text

            routetp = item.findAll('routetp')
            if routetp == []:
                routetp = None
            else:
                routetp = routetp[0].text

            startnodenm = item.findAll('startnodenm')
            if startnodenm == []:
                startnodenm = None
            else:
                startnodenm = startnodenm[0].text

            startvehicletime = item.findAll('startvehicletime')
            if startvehicletime == []:
                startvehicletime = None
            else:
                startvehicletime = startvehicletime[0].text

            endnodenm = item.findAll('endnodenm')
            if endnodenm == []:
                endnodenm = None
            else:
                endnodenm = endnodenm[0].text

            endvehicletime = item.findAll('endvehicletime')
            if endvehicletime == []:
                endvehicletime = None
            else:
                endvehicletime = endvehicletime[0].text
                
            sql = "INSERT INTO public_busroute_info (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime,citycode) VALUES (?,?,?,?,?,?,?,?)"
            val = (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime, location[0])


            cursor.execute(sql, val)
            mydb.commit()

location : 33360 | 1번째 조회시작
location : 33360 | 2번째 조회시작
location : 33360 | 3번째 조회시작
location : 33360 | 4번째 조회시작
location : 33360 | 5번째 조회시작
location : 33360 | 6번째 조회시작
location : 33360 | 7번째 조회시작
location : 33360 | 8번째 조회시작
location : 33360 | 9번째 조회시작
location : 33360 | 10번째 조회시작
location : 33360 | 11번째 조회시작
location : 33360 | 12번째 조회시작
location : 33360 | 13번째 조회시작
location : 33360 | 14번째 조회시작
location : 33360 | 15번째 조회시작
location : 33360 | 16번째 조회시작
location : 33360 | 17번째 조회시작
location : 33360 | 18번째 조회시작
location : 33360 | 19번째 조회시작
location : 33360 | 20번째 조회시작
location : 33360 | 21번째 조회시작
location : 33360 | 22번째 조회시작
location : 33360 | 23번째 조회시작
location : 33360 | 24번째 조회시작
location : 33360 | 25번째 조회시작
location : 33360 | 26번째 조회시작
location : 33360 | 27번째 조회시작
location : 33360 | 28번째 조회시작
location : 33360 | 29번째 조회시작
location : 33360 | 30번째 조회시작
location : 33360 | 31번째 조회시작
location : 33360 | 32번째 조회시작
location : 33360 | 33번째 조회시작
location : 33360 | 34번째 조회시작
location : 33360 | 35번째

103 번쨰데이터가없습니다.
location : 33370 | 104번째 조회시작
104 번쨰데이터가없습니다.
location : 33370 | 105번째 조회시작
105 번쨰데이터가없습니다.
location : 33370 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 33370 | 107번째 조회시작
107 번쨰데이터가없습니다.
location : 33370 | 108번째 조회시작
108 번쨰데이터가없습니다.
location : 33370 | 109번째 조회시작
109 번쨰데이터가없습니다.
location : 33370 | 110번째 조회시작
110 번쨰데이터가없습니다.
location : 33370 | 111번째 조회시작
111 번쨰데이터가없습니다.
location : 33370 | 112번째 조회시작
112 번쨰데이터가없습니다.
location : 33370 | 113번째 조회시작
113 번쨰데이터가없습니다.
location : 33370 | 114번째 조회시작
114 번쨰데이터가없습니다.
location : 33370 | 115번째 조회시작
115 번쨰데이터가없습니다.
location : 33370 | 116번째 조회시작
116 번쨰데이터가없습니다.
location : 33370 | 117번째 조회시작
117 번쨰데이터가없습니다.
location : 33370 | 118번째 조회시작
118 번쨰데이터가없습니다.
location : 33370 | 119번째 조회시작
119 번쨰데이터가없습니다.
location : 33370 | 120번째 조회시작
location : 33380 | 1번째 조회시작
location : 33380 | 2번째 조회시작
location : 33380 | 3번째 조회시작
location : 33380 | 4번째 조회시작
location : 33380 | 5번째 조회시작
location : 33380 | 6번째 조회시작
location : 33380 | 7번째 조회시작
location : 33380 | 8번째

96 번쨰데이터가없습니다.
location : 34010 | 97번째 조회시작
location : 34010 | 98번째 조회시작
location : 34010 | 99번째 조회시작
location : 34010 | 100번째 조회시작
location : 34010 | 101번째 조회시작
location : 34010 | 102번째 조회시작
location : 34010 | 103번째 조회시작
location : 34010 | 104번째 조회시작
location : 34010 | 105번째 조회시작
location : 34010 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 34010 | 107번째 조회시작
107 번쨰데이터가없습니다.
location : 34010 | 108번째 조회시작
108 번쨰데이터가없습니다.
location : 34010 | 109번째 조회시작
109 번쨰데이터가없습니다.
location : 34010 | 110번째 조회시작
location : 34010 | 111번째 조회시작
111 번쨰데이터가없습니다.
location : 34010 | 112번째 조회시작
location : 34010 | 113번째 조회시작
location : 34010 | 114번째 조회시작
location : 34010 | 115번째 조회시작
location : 34010 | 116번째 조회시작
location : 34010 | 117번째 조회시작
location : 34010 | 118번째 조회시작
location : 34010 | 119번째 조회시작
location : 34010 | 120번째 조회시작
location : 34020 | 1번째 조회시작
location : 34020 | 2번째 조회시작
location : 34020 | 3번째 조회시작
location : 34020 | 4번째 조회시작
location : 34020 | 5번째 조회시작
location : 34020 | 6번째 조회시작
location : 34020 | 7

92 번쨰데이터가없습니다.
location : 34040 | 93번째 조회시작
93 번쨰데이터가없습니다.
location : 34040 | 94번째 조회시작
94 번쨰데이터가없습니다.
location : 34040 | 95번째 조회시작
95 번쨰데이터가없습니다.
location : 34040 | 96번째 조회시작
96 번쨰데이터가없습니다.
location : 34040 | 97번째 조회시작
location : 34040 | 98번째 조회시작
location : 34040 | 99번째 조회시작
location : 34040 | 100번째 조회시작
location : 34040 | 101번째 조회시작
location : 34040 | 102번째 조회시작
102 번쨰데이터가없습니다.
location : 34040 | 103번째 조회시작
103 번쨰데이터가없습니다.
location : 34040 | 104번째 조회시작
104 번쨰데이터가없습니다.
location : 34040 | 105번째 조회시작
105 번쨰데이터가없습니다.
location : 34040 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 34040 | 107번째 조회시작
107 번쨰데이터가없습니다.
location : 34040 | 108번째 조회시작
108 번쨰데이터가없습니다.
location : 34040 | 109번째 조회시작
109 번쨰데이터가없습니다.
location : 34040 | 110번째 조회시작
110 번쨰데이터가없습니다.
location : 34040 | 111번째 조회시작
111 번쨰데이터가없습니다.
location : 34040 | 112번째 조회시작
112 번쨰데이터가없습니다.
location : 34040 | 113번째 조회시작
113 번쨰데이터가없습니다.
location : 34040 | 114번째 조회시작
114 번쨰데이터가없습니다.
location : 34040 | 115번째 조회시작
115 번쨰데이터가없습니다.
location : 34040 |

84 번쨰데이터가없습니다.
location : 34060 | 85번째 조회시작
85 번쨰데이터가없습니다.
location : 34060 | 86번째 조회시작
86 번쨰데이터가없습니다.
location : 34060 | 87번째 조회시작
87 번쨰데이터가없습니다.
location : 34060 | 88번째 조회시작
88 번쨰데이터가없습니다.
location : 34060 | 89번째 조회시작
89 번쨰데이터가없습니다.
location : 34060 | 90번째 조회시작
90 번쨰데이터가없습니다.
location : 34060 | 91번째 조회시작
91 번쨰데이터가없습니다.
location : 34060 | 92번째 조회시작
92 번쨰데이터가없습니다.
location : 34060 | 93번째 조회시작
93 번쨰데이터가없습니다.
location : 34060 | 94번째 조회시작
94 번쨰데이터가없습니다.
location : 34060 | 95번째 조회시작
95 번쨰데이터가없습니다.
location : 34060 | 96번째 조회시작
96 번쨰데이터가없습니다.
location : 34060 | 97번째 조회시작
97 번쨰데이터가없습니다.
location : 34060 | 98번째 조회시작
98 번쨰데이터가없습니다.
location : 34060 | 99번째 조회시작
99 번쨰데이터가없습니다.
location : 34060 | 100번째 조회시작
100 번쨰데이터가없습니다.
location : 34060 | 101번째 조회시작
location : 34060 | 102번째 조회시작
location : 34060 | 103번째 조회시작
103 번쨰데이터가없습니다.
location : 34060 | 104번째 조회시작
location : 34060 | 105번째 조회시작
105 번쨰데이터가없습니다.
location : 34060 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 34060 | 107번째 조회시작
107 번쨰데이터가없습니다.
locat